In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
import warnings
warnings.filterwarnings("ignore")
rcParams['figure.figsize'] = 15, 6

In [2]:
data = pd.read_csv('/home/cloudera/FeedDB_backup.csv',sep=',',index_col='day',parse_dates=[0])
print(data.info())
print ('\n Data Types:')
print (data.dtypes)

In [3]:
print(data.shape[0])
print(data.index)
data.head()
#data.count

In [4]:
# this is final DataTime series (Datatime formart must be yyyy-mm-dd)
ts= data
print(ts.describe())
ts['count']['2017-05-02']

In [5]:
#from pandas import Series
from matplotlib import pyplot  as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

In [6]:
plt.plot(ts)
#plt.xticks(rotation='vertical')
plt.show()
from pandas.tools.plotting import autocorrelation_plot
autocorrelation_plot(ts)
plt.show()

In [7]:
#transform here for simplicity

In [8]:
'''Dickey-Fuller Test

How to Check Stationarity of a Time Series?
'''

In [9]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries,size):
    print(timeseries.head(5))
    #Determing rolling statistics
    rolmean = pd.rolling_mean(timeseries, window=size)
    rolstd = pd.rolling_std(timeseries, window=size)

    #Plot rolling statistics:
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

In [10]:
#series.head(3)
test_stationarity(ts['count'],6)

In [11]:
'''Note:- VVIP the test statistic is way more than the critical values. Note that the signed values should be compared and not the absolute values.

TS towards stationarity. exponentially weighted moving average
'''

In [12]:
'''Estimating & Eliminating Trend'''

In [13]:
ts_log = np.log(ts)
plt.plot(ts_log)
# other options are taking a log, square root, cube root, etc

In [14]:
'''how to remove trend from data :-some of most commonly used are:

Aggregation – taking average for a time period like monthly/weekly averages
Smoothing – taking rolling averages
Polynomial Fitting – fit a regression model
'''

In [15]:
#Moving average

In [16]:
moving_avg_10 = pd.rolling_mean(ts_log['2017-09-05':],10)
moving_avg_7 = pd.rolling_mean(ts_log['2017-09-05':],7)
moving_avg_4 = pd.rolling_mean(ts_log['2017-09-05':],4)
plt.plot(ts_log['2017-09-05':])
plt.plot(moving_avg_10, color='red')
plt.plot(moving_avg_7, color='green')
plt.plot(moving_avg_4, color='black')

In [17]:
moving_avg_10 = pd.rolling_mean(ts_log['2017-10-09':],10)
moving_avg_7 = pd.rolling_mean(ts_log['2017-10-09':],7)
moving_avg_4 = pd.rolling_mean(ts_log['2017-10-09':],4)
plt.plot(ts_log['2017-11-09':])
plt.plot(moving_avg_10, color='red')
plt.plot(moving_avg_7, color='green')
plt.plot(moving_avg_4, color='black')

In [18]:
moving_avg = pd.rolling_mean(ts_log,4)
plt.plot(ts_log)
plt.plot(moving_avg, color='red')

In [19]:
ts_log_moving_avg_diff.dropna(inplace=True)
test_stationarity(ts_log_moving_avg_diff['count'],4)

In [20]:
expwighted_avg = pd.ewma(ts_log, halflife=4)
plt.plot(ts_log)
plt.plot(expwighted_avg, color='red')

In [21]:
ts_log_ewma_diff[ts_log_ewma_diff['count'] == 0].count()

In [22]:
ts_log_ewma_diff = ts_log - expwighted_avg
ts_log_ewma_diff.dropna(inplace=True)
ts_log_ewma_diff = ts_log_ewma_diff[ts_log_ewma_diff['count'] < 0]
test_stationarity(ts_log_ewma_diff['count'],4)
#the test statistic is smaller than the 1% critical value, which is better than the previous case.

In [23]:
'''Eliminating Trend and Seasonality

The simple trend reduction techniques discussed before don’t work in all cases, particularly the ones with high seasonality. Lets discuss two ways of removing trend and seasonality:

Differencing – taking the differece with a particular time lag
Decomposition – modeling both trend and seasonality and removing them from the model.

Differencing

One of the most common methods of dealing with both trend and seasonality is differencing. In this technique, we take the difference of the observation at a particular instant with that at the previous instant. This mostly works well in improving stationarity. First order differencing can be done in Pandas as:
'''

In [24]:
ts_log_diff = ts_log - ts_log.shift()
plt.plot(ts_log_diff)

In [25]:
ts_log_diff.dropna(inplace=True)
test_stationarity(ts_log_diff['count'],4)

In [26]:
'''Decomposing

In this approach, both trend and seasonality are modeled separately and the remaining part of the series is returned.

from statsmodels.tsa.seasonal import seasonal_decompose decomposition = seasonal_decompose(ts_log,freq=12)

trend = decomposition.trend seasonal = decomposition.seasonal residual = decomposition.resid

plt.subplot(411) plt.plot(ts_log, label='Original') plt.legend(loc='best') plt.subplot(412) plt.plot(trend, label='Trend') plt.legend(loc='best') plt.subplot(413) plt.plot(seasonal,label='Seasonality') plt.legend(loc='best') plt.subplot(414) plt.plot(residual, label='Residuals') plt.legend(loc='best') plt.tight_layout()

ts_log_decompose = residual ts_log_decompose.dropna(inplace=True) test_stationarity(ts_log_decompose,4)

Auto-Regressive Integrated Moving Averages. The ARIMA forecasting for a stationary time series is nothing but a linear (like a linear regression) equation. The predictors depend on the parameters (p,d,q) of the ARIMA model:

The predictors depend on the parameters (p,d,q) of the ARIMA model:

Number of AR (Auto-Regressive) terms (p): AR terms are just lags of dependent variable. For instance if p is 5, the predictors for x(t) will be x(t-1)….x(t-5).
Number of MA (Moving Average) terms (q): MA terms are lagged forecast errors in prediction equation. For instance if q is 5, the predictors for x(t) will be e(t-1)….e(t-5) where e(i) is the difference between the moving average at ith instant and actual value.
Number of Differences (d): These are the number of nonseasonal differences, i.e. in this case we took the first order difference. So either we can pass that variable and put d=0 or pass the original variable and put d=1. Both will generate same results.

An importance concern here is how to determine the value of ‘p’ and ‘q’. We use two plots to determine these numbers. Lets discuss them first.

Autocorrelation Function (ACF): It is a measure of the correlation between the the TS with a lagged version of itself. For instance at lag 5, ACF would compare series at time instant ‘t1’…’t2’ with series at instant ‘t1-5’…’t2-5’ (t1-5 and t2 being end points).
Partial Autocorrelation Function (PACF): This measures the correlation between the TS with a lagged version of itself but after eliminating the variations already explained by the intervening comparisons. Eg at lag 5, it will check the correlation but remove the effects already explained by lags 1 to 4.

The ACF and PACF plots for the TS after differencing can be plotted as:
'''

In [27]:
#ACF and PACF plots:
from statsmodels.tsa.stattools import acf, pacf

In [28]:
lag_acf = acf(ts_log_moving_avg_diff, nlags=4)
lag_pacf = pacf(ts_log_moving_avg_diff, nlags=4, method='ols')

In [29]:
plt.subplot(411)
plt.axhline(y=0,linestyle='--',color='red')
plt.plot(lag_pacf, label='pacf')
plt.legend(loc='pacf')
plt.subplot(412)
plt.axhline(y=0,linestyle='--',color='red')
plt.plot(lag_acf, label='acf')
plt.legend(loc='acf')

In [30]:
#Plot ACF: 
plt.subplot(121) 
plt.plot(lag_acf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts_log_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts_log_diff)),linestyle='--',color='gray')
plt.title('Autocorrelation Function')

In [31]:
#Plot PACF:
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts_log_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts_log_diff)),linestyle='--',color='gray')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()

In [32]:
'''p – The lag value where the PACF chart crosses the upper confidence interval for the first time. If you notice closely, in this case p=2.
q – The lag value where the ACF chart crosses the upper confidence interval for the first time. If you notice closely, in this case q=2.

Now, lets make 3 different ARIMA models considering individual as well as combined effects. I will also print the RSS for each. Please note that here RSS is for the values of residuals and not actual series.
'''

In [33]:
#We need to load the ARIMA model first:
from statsmodels.tsa.arima_model import ARIMA
# The p,d,q values can be specified using the order argument of ARIMA which take a tuple (p,d,q). Let model the 3 cases:

In [34]:
#AR Model
model = ARIMA(ts_log_moving_avg_diff, order=(2, 0, 0))  # for this i am getting RSS is Nan order=(2, 1, 0))
results_AR = model.fit(disp=0)  
plt.plot(ts_log_moving_avg_diff)
plt.plot(results_AR.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_AR.fittedvalues-ts_log_moving_avg_diff['count'])**2))
results_AR.summary()

In [35]:
#MA Model

model = ARIMA(ts_log_moving_avg_diff, order=(0, 0, 2))  
results_MA = model.fit(disp=-1)  
plt.plot(ts_log_moving_avg_diff)
plt.plot(results_MA.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_MA.fittedvalues-ts_log_moving_avg_diff['count'])**2))

In [36]:
#Combined Model
model = ARIMA(ts_log_moving_avg_diff, order=(2, 0, 2))  
results_ARIMA = model.fit(disp=-1)  
plt.plot(ts_log_moving_avg_diff)
plt.plot(results_ARIMA.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_ARIMA.fittedvalues-ts_log_moving_avg_diff['count'])**2))

In [37]:
# plot residual
residual = pd.DataFrame(results_ARIMA.resid)
residual.plot()
plt.show()

In [38]:
residual.plot(kind='kde')
plt.show()
print(residual.describe())

In [39]:
predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues, copy=True)
print (predictions_ARIMA_diff.head())
plt.plot(ts_log_diff)
plt.plot(predictions_ARIMA_diff, color='red')

In [40]:
#The way to convert the differencing to log scale is to add these differences consecutively to the base number. 
#An easy way to do it is to first determine the cumulative sum at index and then add it to the base number. 
#The cumulative sum can be found as:

predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
print (predictions_ARIMA_diff_cumsum.head())

In [41]:
predictions_ARIMA_log = pd.Series(ts_log.ix[0], index=ts_log.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum,fill_value=0)
predictions_ARIMA_log.head()

In [42]:
predictions_ARIMA = np.exp(predictions_ARIMA_log)
plt.plot(ts)
plt.plot(predictions_ARIMA)
#plt.title('RMSE: %.4f'% np.sqrt(sum((predictions_ARIMA-ts)**2)/len(ts)))